# First pass code: use only pre-tournament season-long advanced stats

# Import data

In [218]:
# Imports
import numpy as np
import pandas as pd 
#from sportsreference.ncaab.teams import Teams
from tqdm import tqdm
import os
import difflib

from sklearn.linear_model import LogisticRegression

In [306]:
# Import data

FILE_PATH = os.path.join("..", "mens-march-mania-2022", "MDataFiles_Stage2")
#os.path.join("mens-march-mania-2022", "MDataFiles_Stage1")

df_hist = pd.read_csv(
    os.path.join(FILE_PATH, "MNCAATourneyCompactResults.csv"))
df_names = pd.read_csv( os.path.join(FILE_PATH, "MTeamSpellings.csv"),  encoding= 'unicode_escape')
names_dict = {str(df_names["TeamNameSpelling"][idx]):int(df_names["TeamID"][idx]) for idx in range(len(df_names["TeamID"]))}

res_2022 = pd.read_csv(
    os.path.join(FILE_PATH, "Results_2022.csv"))
res_2022 = remove_WL_data(res_2022)


In [304]:
res_2022.head()

,Year,ATeamID,BTeamID,Result
0,2022,1103,1417,1
1,2022,1104,1323,1
2,2022,1112,1222,1
3,2022,1112,1395,1
4,2022,1112,1460,1


In [300]:
res_2022.head()

,Year,Wteam,Lteam
0,2022,1103,1417
1,2022,1104,1323
2,2022,1112,1222
3,2022,1112,1395
4,2022,1112,1460


## Adding capability to find TeamID for alternate team names

Extends exists TeamNameSpelling file to accurately translate KenPom and 538 names

In [214]:
cardinal_dir = {"west": "w", "western": "w", "north": "n", "northern": "n", "south": "s", "southern": "s", "east": "e", "eastern": "e"}

for key in names_dict.keys(): 
    split_arr = key.split()

    # CSU mods
    if split_arr[0] == "cal" and split_arr[1] == "state": 
        new_item1 = " ".join( ["csu", *split_arr[2:]])
        if new_item1 not in names_dict.keys(): 
            df_names.loc[len(df_names)]= [new_item1, names_dict[key]]

    # Directional school mods
    for idx, part in enumerate(split_arr): 
        if part in cardinal_dir: 
            new_item1 = " ".join( [*split_arr[:idx], cardinal_dir[part], *split_arr[idx+1:] ])
            if new_item1 not in names_dict.keys(): 
                df_names.loc[len(df_names)]= [new_item1, names_dict[key]]
            new_item2 = " ".join( [*split_arr[:idx], cardinal_dir[part] + ".", *split_arr[idx+1:] ])
            if new_item2 not in names_dict.keys(): 
                df_names.loc[len(df_names)]= [new_item2, names_dict[key]]


# A few names that need to be added
df_names.loc[len(df_names)]= ["liu", names_dict["liu brooklyn"]]
df_names.loc[len(df_names)]= ["ark.-lr", names_dict["ark little rock"]]
df_names.loc[len(df_names)]= ["sdsu", names_dict["san diego state"]]
df_names.loc[len(df_names)]= ["miami", names_dict["miami fl"]]
df_names.loc[len(df_names)]= ["jax. state", names_dict["jacksonville state"]]
df_names.loc[len(df_names)]= ["st. bon.", names_dict["st bonaventure"]]
df_names.loc[len(df_names)]= ["chatt.", names_dict["chattanooga"]]
df_names.loc[len(df_names)]= ["app st.", names_dict["appalachian st"]]
df_names.loc[len(df_names)]= ["miss. st.", names_dict["mississippi st"]]


df_names.to_csv( os.path.join(FILE_PATH, "MTeamSpellingsExtended.csv"))
names_dict = {str(df_names["TeamNameSpelling"][idx]):int(df_names["TeamID"][idx]) for idx in range(len(df_names["TeamID"]))}


## Cleaning data

In [302]:
# Function to build larger df from a given 
def remove_WL_data(df, add_res = True):
    w_col = []
    l_col = []


    w_df= df.copy()
    l_df = df.copy()
    # Removing win/loss naming
    for col in df.columns:
        if col[0] == "W":
            l_val = "B" + col[1:]
            w_val = "A" + col[1:]
        elif col[0] == "L":
            l_val = "A" + col[1:]
            w_val = "B" + col[1:]
        else:
            l_val = col
            w_val = col

        w_col.append(w_val) 
        l_col.append(l_val) 
    
    w_df.columns = w_col
    l_df.columns = l_col

    # Show that team A won in w_df and lost in l_df
    if add_res:
        
        w_df["Result"] = 1
        l_df["Result"] = 0
    
    if "ALoc" in df.columns: 
        l_df["ALoc"] = "N"
        a_home_idx = l_df[l_df["BLoc"]=="A"].index
        l_df.loc["ALoc"][a_home_idx] = "H"
        a_away_idx = l_df[l_df["BLoc"]=="H"].index
        l_df["ALoc"][a_away_idx] = "A"
        l_df.drop(columns=["BLoc"], inplace=True)

    return pd.merge(w_df, l_df, how="outer")



## Importing 538 and Kenpom pre-tournament data

In [221]:
def add_year_data(path_str, og_df, year, names_dict, verbose = False): 
    """
    Import data from CSV, add the current year, 
    and concat with existing df
    """

    df = pd.read_csv(
        os.path.join(FILE_PATH, path_str + str(year) + ".csv"))
    
    # Adding column showing year and initializing column for TeamID
    df[["Year", "TeamID"]] = (year,0)


    # Remove unnamed columns and blank rows
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    df = df.dropna() 
    drop_list = []

    # Removing seeds from team names
    for idx in df.index:
        # If value is to be dropped, save and skip for future removal
        if df["Team"][idx] == "NaN" or df["Team"][idx] =="Team":
            drop_list.append(idx)
            continue
        else: 
            num_check = df["Team"][idx].split()
            
            # Remove numbers following team names (ie seeds)
            if num_check[-1].isnumeric(): 
                df.loc[ idx, "Team"] = " ".join(num_check[0:-1])

        
        # Find Team ID 
        a = str(df.loc[idx, "Team"]).strip()
        
        # If exact match exists, use associated team id
        try:
            df.loc[idx, "TeamID"] = names_dict[a.lower()]
        # Otherwise, find closest match
        except: 
            approx_name = difflib.get_close_matches(a.lower(), names_dict.keys())[0]
            if verbose: 
                print("Replacting " + str(a.lower()) + " with " + str(approx_name))
            df.loc[idx, "TeamID"] = names_dict[approx_name]

    # Remove values of na/team names
    df = df.drop(drop_list)
        
    return pd.concat([og_df, df], ignore_index=True)

# Importing and cleaning ranking data

In [223]:
f38_str = "Mens_538_"
kp_str = "Mens_Kenpom_"
kp_strt = "Mens_Kenpomt_"
f38_df = pd.DataFrame()
kp_df = pd.DataFrame()

for year in range(2016, 2022):
    if year != 2020: 
        f38_df = add_year_data(f38_str, f38_df, year, names_dict)
        kp_df = add_year_data(kp_str, kp_df, year, names_dict)


# Cleaning tournament data

In [237]:
df_hist_short = df_hist[df_hist["Season"] >  2015]
df_hist_short.reset_index(drop=True, inplace=True)

df_hist_short = remove_WL_data(df_hist_short)
print(df_hist_short.head())

   Season  DayNum  ATeamID  AScore  BTeamID  BScore ALoc  NumOT  Result
0    2016     134     1195      96     1192      65    N      0       1
1    2016     134     1455      70     1435      50    N      0       1
2    2016     135     1221      59     1380      55    N      0       1
3    2016     135     1276      67     1409      62    N      0       1
4    2016     136     1114      85     1345      83    N      2       1


C:\Users\paul\AppData\Local\Temp\ipykernel_23700\1134114374.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_df["ALoc"][a_home_idx] = "H"
C:\Users\paul\AppData\Local\Temp\ipykernel_23700\1134114374.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_df["ALoc"][a_away_idx] = "A"


## Merging in KenPom data

In [271]:
kp_df_short.describe()

,Year,TeamID
count,1762.000000,1762.000000
mean,2018.210556,1284.801930
std,1.725462,104.431241
min,2016.000000,1101.000000
25%,2017.000000,1195.000000
50%,2018.000000,1284.000000
75%,2019.000000,1375.000000
max,2021.000000,1471.000000


In [305]:
merged_df_0 = df_hist_short

kp_df_short =  kp_df.drop(columns=["Rank", "Team", "Conf", "W-L", "AdjEM", "AdjEM.1", "AdjEM.2", "Rk"])
kp_df_short["AdjEM_Oppo"] = kp_df_short["OppO"].apply(lambda x: float(x)) -kp_df_short["OppD"].apply(lambda x: float(x))

def merge_KP_and_prep(merged_df_0, kp_df_short): 
    kp_cols = kp_df_short.columns
    for str1 in ["A", "B"]:
        

        # Updating columns to match
        
        kp_cols_a = [str1 + name_val for name_val in kp_cols]
        kp_cols_a[kp_cols_a.index(str1 + "Year")] = "Season"

        kp_df_short.columns = kp_cols_a
        merged_df_0 = merged_df_0.merge(kp_df_short, how="inner", on = [str1 + "TeamID", "Season"])
    return merged_df_0
merged_df_0 = merge_KP(merged_df_0, kp_df_short)

In [282]:
merged_df_0.head()

,Season,DayNum,ATeamID,AScore,BTeamID,BScore,ALoc,NumOT,Result,APyth,...,BPyth,BAdjO,BAdjD,BAdjT,BLuck,BPyth.1,BOppO,BOppD,BPyth.2,BAdjEM_Oppo
0,2016,134,1195,96,1192,65,N,0,1,0.4686,...,0.2904,105.5,114.1,72,0.092,0.3147,98.9,105.8,0.5596,-6.9
1,2016,136,1195,67,1314,83,N,0,0,0.4686,...,0.9407,119.5,94,72,-0.043,0.721,109.3,100.6,0.5742,8.7
2,2016,144,1231,86,1314,101,N,0,0,0.8959,...,0.9407,119.5,94,72,-0.043,0.721,109.3,100.6,0.5742,8.7
3,2016,138,1344,66,1314,85,N,0,0,0.7991,...,0.9407,119.5,94,72,-0.043,0.721,109.3,100.6,0.5742,8.7
4,2016,146,1323,74,1314,88,N,0,0,0.8131,...,0.9407,119.5,94,72,-0.043,0.721,109.3,100.6,0.5742,8.7


In [295]:
simple_features = [ "AAdjO", "AAdjD", "AAdjT", "AAdjEM_Oppo", "BAdjO", "BAdjD", "BAdjT", "BAdjEM_Oppo", ]

extracted_df= (merged_df_0[["Result", *simple_features]]).astype(float)
extracted_df.describe()

,Result,AAdjO,AAdjD,AAdjT,AAdjEM_Oppo,BAdjO,BAdjD,BAdjT,BAdjEM_Oppo
count,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000
mean,0.500000,113.959281,95.869162,67.720958,6.340868,113.959281,95.869162,67.720958,6.340868
std,0.500375,6.118930,5.121540,2.977554,6.343495,6.118930,5.121540,2.977554,6.343495
min,0.000000,96.100000,84.400000,58.600000,-12.700000,96.100000,84.400000,58.600000,-12.700000
25%,0.000000,110.000000,92.700000,65.700000,2.575000,110.000000,92.700000,65.700000,2.575000
50%,0.500000,114.100000,95.700000,67.800000,8.200000,114.100000,95.700000,67.800000,8.200000
75%,1.000000,118.900000,99.000000,69.700000,10.900000,118.900000,99.000000,69.700000,10.900000
max,1.000000,127.400000,114.100000,76.600000,18.900000,127.400000,114.100000,76.600000,18.900000


# Training on tournament data


In [292]:
def normalize_data(input_df, ingore_arr=["Result"]): 
    df = input_df.copy()
    for col in df.columns: 
        if col not in ingore_arr: 
            df[col] = (df[col] -np.min(df[col]))/( np.max(df[col]) - np.min(df[col]))
    return df

In [293]:
#extracted_df.to_csv("extracted_check.csv")

a= extracted_df.dropna()
a.describe()

,Result,AAdjO,AAdjD,AAdjT,AAdjEM_Oppo,BAdjO,BAdjD,BAdjT,BAdjEM_Oppo
count,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000
mean,0.500000,113.959281,95.869162,67.720958,6.340868,113.959281,95.869162,67.720958,6.340868
std,0.500375,6.118930,5.121540,2.977554,6.343495,6.118930,5.121540,2.977554,6.343495
min,0.000000,96.100000,84.400000,58.600000,-12.700000,96.100000,84.400000,58.600000,-12.700000
25%,0.000000,110.000000,92.700000,65.700000,2.575000,110.000000,92.700000,65.700000,2.575000
50%,0.500000,114.100000,95.700000,67.800000,8.200000,114.100000,95.700000,67.800000,8.200000
75%,1.000000,118.900000,99.000000,69.700000,10.900000,118.900000,99.000000,69.700000,10.900000
max,1.000000,127.400000,114.100000,76.600000,18.900000,127.400000,114.100000,76.600000,18.900000


In [294]:
norm_df = normalize_data(extracted_df)

In [291]:
norm_df.describe()

,Result,AAdjO,AAdjD,AAdjT,AAdjEM_Oppo,BAdjO,BAdjD,BAdjT,BAdjEM_Oppo
count,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000
mean,0.500000,0.570584,0.386167,0.506720,0.602559,0.570584,0.386167,0.506720,0.602559
std,0.500375,0.195493,0.172442,0.165420,0.200744,0.195493,0.172442,0.165420,0.200744
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.444089,0.279461,0.394444,0.483386,0.444089,0.279461,0.394444,0.483386
50%,0.500000,0.575080,0.380471,0.511111,0.661392,0.575080,0.380471,0.511111,0.661392
75%,1.000000,0.728435,0.491582,0.616667,0.746835,0.728435,0.491582,0.616667,0.746835
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [298]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
x_train = norm_df[simple_features]
y_train = norm_df["Result"]
logisticRegr.fit(x_train, y_train)

LogisticRegression()

In [ ]:
predict_2022 = 

# Find ranking values for each week in each team-season


In [3]:
def get_rankings(rank_identifier, mass_df):
    """Get a dataframe with for the given rank_identifier """
    days = np.unique(mass_df["RankingDayNum"])
    teams =  np.unique(mass_df["TeamID"]) 


    ident_idx = mass_df[mass_df["SystemName"]!=rank_identifier].index
    ident_df = mass_df.drop(ident_idx)

    indent_rank_df = pd.DataFrame(columns=teams, index=days, )

    for idx in ident_df.index: 
        indent_rank_df.at[ident_df["RankingDayNum"][idx], ident_df["TeamID"][idx] ]= ident_df["OrdinalRank"][idx]

    return indent_rank_df


POM_df = get_rankings("POM", mass_df)

In [6]:
def add_rankings(df, rank_df, rank_name): 
    """Adding ranking values of winning and losing teams for df"""

    # Find days in the rankings to pull days from
    rank_idx = np.searchsorted(rank_df.index, df["DayNum"])
    df["WRank"] = rank_df.loc[[rank_idx, df["WTeamID"]], :]
    return df
#add_rankings(df_reg, POM_df, "POM")

In [ ]:
# First pass features
